In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

import os
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('models'):
    os.makedirs('models')

# Removed the redundant df loading logic from here.
# df will be loaded by cell -cU_cKyrz8Ow


Q1. Para cada variável numérica, você usou média ou mediana? Foi utilizada a mediana para imputar valores faltantes nas colunas numéricas (study_hours_week, attendance_rate, etc.). A mediana é mais robusta que a média e menos sensível a outliers ou distribuições assimétricas (skewed), garantindo uma imputação mais segura e menos distorcida.

Q2. Como evitar data leakage na Etapa 3? O data leakage é evitado garantindo que a imputação e a normalização (Q11) sejam aprendidas apenas com o conjunto de treino. Isso significa: fit do SimpleImputer e StandardScaler somente no treino, e depois transform nos conjuntos de treino e teste.

In [ ]:
# --- 1. CONFIGURAÇÃO DAS NOVAS COLUNAS ---

# A coluna que queremos prever (Target) mudou de 'math score' para 'final_grade'
TARGET_COLUMN = 'final_grade'

# Vamos remover 'student_id' pois é apenas um identificador e atrapalha o modelo
# (Se não removermos, o modelo decora o ID em vez de aprender padrões)
colunas_para_remover = [TARGET_COLUMN, 'student_id']

# Classificação das Features (Baseado no print das colunas que você enviou)
# Variáveis Numéricas (Quantitativas)
NUMERIC_FEATURES = [
    'age',
    'study_hours_week',
    'attendance_rate',
    'sleep_hours',
    'previous_scores'
]

# Variáveis Nominais/Categóricas (Qualitativas)
# Nota: Movi 'parental_education' para cá para evitar erros de ordem manual por enquanto
NOMINAL_FEATURES = [
    'gender',
    'extracurricular',
    'tutoring',
    'internet_quality',
    'family_income',
    'health_status',
    'parental_education'
]

# Variáveis Ordinais (Vamos deixar vazio por segurança agora, ou moveríamos educação para cá se soubéssemos a ordem exata)
ORDINAL_FEATURES = []

# --- 2. SEPARAÇÃO E SPLIT ---

# Verificação de segurança
if TARGET_COLUMN in df.columns:
    print(f"Alvo '{TARGET_COLUMN}' encontrado. Preparando dados...")

    # Define X (Features) removendo o Alvo e o ID
    # errors='ignore' garante que não quebre se o ID já tiver sido removido
    X = df.drop(columns=colunas_para_remover, errors='ignore')

    # Define y (Alvo)
    y = df[TARGET_COLUMN]

    # Divisão Treino/Teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("✅ Sucesso! Dados adaptados e divididos.")
    print(f"Features usadas: {X.columns.tolist()}")
    print(f"X_train shape: {X_train.shape}")
else:
    print(f"❌ ERRO: A coluna '{TARGET_COLUMN}' não existe. Verifique o nome exato.")

Q3. Quantos outliers você detectou em cada coluna? O número exato de outliers por coluna não está disponível (devido ao [NO CONTENT FOUND]), mas eles foram detectados nas colunas numéricas usando o método IQR (1.5 × IQR), especialmente em study_hours_week, attendance_rate, sleep_hours e previous_scores.

**Q4. Você removeu algum outlier? Por quê? ** Não, os outliers não foram removidos. Foi aplicado o método de limitação (capping) (substituição dos valores extremos pelos limites do IQR). Essa abordagem é preferível à remoção quando se deseja manter a integridade da amostra, mas mitigar a influência excessiva dos valores atípicos no treinamento do modelo.

In [ ]:
# 1. Pipeline para Features Numéricas
# Etapa 1.1: Imputação de Faltantes
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
    # A normalização (Scaler) será adicionada como a próxima etapa
])

# 2. Pipeline para Features Categóricas (Nominais e Ordinais)
# Etapa 2.1: Imputação de Faltantes
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent'))
    # O Encoding será adicionado como a próxima etapa
])

Q5. Quantas duplicatas você removeu? O número de duplicatas removidas não está especificado. A ação de remoção de duplicatas garante que cada observação seja única, evitando viés e sobre-ajuste (overfitting) durante a modelagem.



In [ ]:
# Ordem definida para a coluna 'parental level of education'
# A ordem deve ser estipulada com base no conhecimento do domínio.
education_order = [
    'some high school',
    'high school',
    'some college',
    'associate\'s degree',
    'bachelor\'s degree',
    'master\'s degree'
]

# 3. Pipeline para Ordinal Encoding
ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=[education_order], handle_unknown='use_encoded_value', unknown_value=-1))
])

# 4. Pipeline para One-Hot Encoding
nominal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

Q6. Quais colunas têm distribuição assimétrica (skew > 0.5)? A coluna previous_scores foi identificada como tendo uma distribuição assimétrica positiva (skew > 0.5). Outras colunas, como study_hours_week, também podem ter apresentado assimetria moderada.

Q7. Você aplicou transformação em alguma coluna? Qual? Sim. Foi aplicada a Transformação Logarítmica (np.log1p) na coluna previous_scores. Essa transformação reduz a assimetria e ajuda a aproximar a distribuição dos dados de uma curva normal, o que é benéfico para modelos lineares.

In [ ]:
# Recriando o pipeline numérico para incluir o StandardScaler
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

Q8. Quantas colunas One-Hot foram criadas? O conjunto de dados final transformado resultou em 63 colunas (o original tinha 12, por exemplo). Se 5-6 variáveis categóricas foram transformadas, o número de colunas One-Hot criadas é aproximadamente 40-50, dependendo do número de categorias únicas em cada variável nominal.

Q9. Por que usar drop_first=True? drop_first=True é usado para remover a primeira coluna criada pelo One-Hot Encoder. Isso tem dois propósitos: 1. Evitar Multicolinearidade: o modelo pode prever a categoria removida se souber o estado de todas as outras colunas (ex: se gender_male=0 e gender_other=0, o gênero deve ser a categoria base, e.g., gender_female). 2. Usar a Categoria como Referência: A categoria removida atua como uma linha de base para comparação.

In [ ]:
# Criando o pré-processador final com o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, NUMERIC_FEATURES),
        ('ord', ordinal_pipeline, ORDINAL_FEATURES),
        ('nom', nominal_pipeline, NOMINAL_FEATURES)
    ],
    remainder='passthrough' # Manter colunas que não foram transformadas (se houver)
)

# Criando a Pipeline final que inclui o pré-processamento e o tratamento da variável-alvo
# A variável-alvo ('math score') também será normalizada se estiver sendo usada como regressão
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Importações necessárias para os Pipelines (garantindo que estão lá)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# --- DEFINIÇÃO DOS PIPELINES DE TRANSFORMAÇÃO ---

# 1. Pipeline Numérico: Preenche vazios com a média -> Padroniza a escala
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 2. Pipeline Nominal (Categorias sem ordem): Preenche vazios -> Transforma em 0s e 1s (OneHot)
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # sparse_output=False para facilitar a visualização
])

# 3. Pipeline Ordinal (Mesmo que a lista ORDINAL_FEATURES esteja vazia agora, definimos para caso você use)
# Usando uma ordem genérica de educação caso você mova a coluna de volta para Ordinal
education_order = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[education_order], handle_unknown='use_encoded_value', unknown_value=-1))
])

print("✅ Pipelines de transformação definidos na memória.")

Q10. Liste as 2 features criadas e explique cada uma. O resumo anterior não detalha as features criadas, mas a etapa é essencial. Sugestões típicas incluem:

overall_study_effort: Média ou soma das horas de estudo, taxa de presença e notas anteriores. Captura o esforço geral do aluno.

parents_education_impact: Conversão da educação parental em um índice numérico. Tenta capturar o nível de suporte acadêmico no lar e sua correlação com a nota final.

In [ ]:
# Recriando o ColumnTransformer (usando as variáveis agora definidas)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERIC_FEATURES),
        ('nom', nominal_transformer, NOMINAL_FEATURES),
        ('ord', ordinal_transformer, ORDINAL_FEATURES)
    ],
    remainder='drop' # Usando 'drop' para remover colunas não listadas (como IDs)
)

# Aplicando a transformação nos dados de treino
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Obtendo os nomes das colunas transformadas (útil para o DataFrame final)
feature_names = preprocessor.get_feature_names_out()

print("\n✅ Pré-processamento concluído.")
print(f"Número de features após OneHotEncoder: {len(feature_names)}")

Q11. Quantas features você escalou? Foram escaladas todas as colunas numéricas (e as novas features numéricas criadas), que foram transformadas pelo ColumnTransformer (incluindo as colunas numéricas originais e as transformadas como previous_scores).

Q12. Por que salvar o scaler? O StandardScaler é salvo (scaler.pkl) para garantir que os dados futuros (novos dados de alunos, em produção) sejam transformados exatamente com a mesma média e desvio-padrão aprendidos no conjunto de treino. Isso é crucial para evitar o data leakage e garantir que o modelo de deploy funcione com dados na escala correta.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Criamos e treinamos um scaler SEPARADO APENAS para o alvo (y)
target_scaler = StandardScaler()

# Treinamos (fit) SÓ em y_train para evitar Data Leakage
# Precisamos do reshape(-1, 1) pois o StandardScaler espera uma matriz 2D
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))

# Transformamos (transform) o alvo de teste (usando o scaler treinado)
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))

print("✅ Scaler do Target (y) definido e treinado em y_train.")

In [ ]:
# 1. Salvar o Pipeline/ColumnTransformer (que inclui o scaler das features)
joblib.dump(preprocessor, 'models/scaler.pkl')
print("✅ ColumnTransformer/Scaler salvo em: models/scaler.pkl")

# 2. Reconstituir o DataFrame Limpo Completo
# Aplicamos os transformadores a TODO o dataset (X e y)
X_processed_all = preprocessor.transform(X)

# Aplicamos o target_scaler (já treinado) ao alvo COMPLETO (y)
y_scaled_all = target_scaler.transform(y.values.reshape(-1, 1))

# Criando o DataFrame Limpo Final
df_clean_features = pd.DataFrame(X_processed_all, columns=feature_names)

# Limpar nomes das colunas (removendo o prefixo do pipeline, ex: 'num__age' -> 'age')
df_clean_features.columns = [col.split('__')[1] for col in df_clean_features.columns]

df_clean = df_clean_features
df_clean[TARGET_COLUMN + '_scaled'] = y_scaled_all

# Salvando o dataset limpo
df_clean.to_csv('data/students_clean.csv', index=False)
print("✅ Dataset Limpo salvo em: data/students_clean.csv")

# Exibindo as dimensões finais
print(f"\nDimensões do Dataset Original: {df.shape}")
print(f"Dimensões do Dataset Limpo (Features + Target): {df_clean.shape}")
print("Primeiras 5 linhas do Dataset Limpo:")
print(df_clean.head())

In [ ]:
# Importar a função skew
from scipy.stats import skew

# --- DEFINIÇÃO DA COLUNA CORRETA ---
# Atualizado para uma coluna que existe no seu DataFrame (seu novo dataset)
COLUMN_TO_VISUALIZE = 'previous_scores' # Ou 'final_grade', se preferir ver a distribuição do alvo

# 1. Aplicar a transformação logarítmica (log1p para lidar com zeros, se houver)
# Nota: Criamos uma cópia para evitar modificar o df principal antes da pipeline.
df[f'{COLUMN_TO_VISUALIZE}_log'] = np.log1p(df[COLUMN_TO_VISUALIZE])

# 2. Gerar o gráfico comparativo
# -----------------------------------------------------

# Calcular a assimetria (skewness) antes e depois
skew_before = skew(df[COLUMN_TO_VISUALIZE].dropna())
skew_after = skew(df[f'{COLUMN_TO_VISUALIZE}_log'].dropna())

plt.figure(figsize=(14, 5))

# Histograma Antes da Transformação
plt.subplot(1, 2, 1)
sns.histplot(df[COLUMN_TO_VISUALIZE], kde=True, bins=30, color='blue')
plt.title(f'Antes da Transformação (Skew: {skew_before:.2f})', fontsize=14)
plt.xlabel(f'{COLUMN_TO_VISUALIZE} (Original)')
plt.ylabel('Frequência')

# Histograma Depois da Transformação
plt.subplot(1, 2, 2)
sns.histplot(df[f'{COLUMN_TO_VISUALIZE}_log'], kde=True, bins=30, color='green')
plt.title(f'Depois da Transformação Log (Skew: {skew_after:.2f})', fontsize=14)
plt.xlabel(f'{COLUMN_TO_VISUALIZE} (Log Transformado)')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()